In [1]:
#train the model and save it
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
import numpy as np
import csv

Using TensorFlow backend.


In [2]:
#form input_texts
input_texts = []
with open('with_date(copy).csv') as csvfile:
    next(csvfile)
    readCSV = csv.reader(csvfile,delimiter=',')
    for row in readCSV:
        text = " ".join(row)
        input_texts.append(text)
input_texts

['0 1 5134 Debit 2339 1 India US USD 0.72 2016-03-22',
 '1 2 5192 Debit 4447 1 Pakistan Germany Euro 0.38 2010-09-03',
 '2 3 5410 Debit 6640 0 India India INR 0.91 2012-06-23',
 '3 4 5297 Debit 6076 0 Pakistan Japan Euro 0.41 2021-07-08',
 '4 5 5141 Debit 4790 1 US India INR 0.68 2022-05-19',
 '5 6 5316 Debit 7583 1 India India INR 0.09 2023-06-06',
 '6 7 5169 Credit 5915 1 Japan China Yen 0.89 2015-01-11',
 '7 8 5028 Debit 5733 0 Pakistan US USD 0.94 2021-02-28',
 '8 9 5242 Debit 776 1 India Germany INR 0.71 2009-11-17',
 '9 10 5360 Debit 8928 1 Pakistan India INR 0.37 2009-09-20',
 '10 11 5335 Credit 3372 0 India US USD 0.53 2012-02-08',
 '11 12 5395 Debit 4982 0 Pakistan Germany Euro 0.35 2025-09-19',
 '12 13 5297 Debit 8683 1 India India INR 0.25 2006-01-21',
 '13 14 5352 Credit 5369 0 Pakistan Japan Euro 0.29 2017-05-25',
 '14 15 5479 Credit 4708 1 US India INR 0.34 2016-03-07',
 '15 16 5131 Credit 5032 0 India India INR 0.65 2022-06-23',
 '16 17 5250 Credit 2872 1 Japan China Yen

In [3]:
#form output_texts
with open('description.txt','r',encoding='utf-8') as f:
    lines =   f.read().split('\n') 
target_texts = []
for line in lines:
    line = 'START_ ' + line + ' _END'
    target_texts.append(line)
target_texts

['START_ Transaction no 1 is a debit transaction,performed on 2016-03-22, from India to US worth 2339 dollars, which is not suspicious with 0.72 probability. _END',
 'START_ Transaction no 2 is a debit transaction,performed on 2010-09-03, from Pakistan to Germany worth 4447 euros, which is not suspicious with 0.38 probability. _END',
 'START_ Transaction no 3 is a debit transaction,performed on 2012-06-23, within India worth 6640 ruppees, which is suspicious with 0.91 probability. _END',
 'START_ Transaction no 4 is a debit transaction,performed on 2021-07-08, from Pakistan to Japan worth 6076 euros, which is suspicious with 0.41 probability. _END',
 'START_ Transaction no 5 is a debit transaction,performed on 2022-05-19, from US to India worth 4790 ruppees, which is not suspicious with 0.68 probability. _END',
 'START_ Transaction no 6 is a debit transaction,performed on 2023-06-06, within India worth 7583 ruppees, which is not suspicious with 0.09 probability. _END',
 'START_ Transac

In [4]:
input_words = set()
target_words = set()
for input_text,target_text in zip(input_texts,target_texts):
    words = input_text.split(' ') 
    for word in words:
        if word not in input_words:
            input_words.add(word)
    words = target_text.split(' ') 
    for word in words:
        if word not in target_words:
            target_words.add(word)
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
input_text_sizes = []
for text in input_texts:
    input_text_sizes.append(len(text.split(' ')))
target_text_sizes = []
for text in target_texts:
    target_text_sizes.append(len(text.split(' ')))
max_encoder_seq_length = max(input_text_sizes)
max_decoder_seq_length = max(target_text_sizes)  

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 15471
Number of unique output tokens: 15487
Max sequence length for inputs: 11
Max sequence length for outputs: 25


In [6]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [7]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, word in enumerate(input_text.split(' ')):
        encoder_input_data[i,t]=input_token_index[word]
    for t, word in enumerate(target_text.split(' ')):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word] 
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [9]:
embedding_size = 50
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

W0918 10:31:54.292426  2328 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0918 10:31:54.338248  2328 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0918 10:31:54.350485  2328 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [10]:
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     773550      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     774350      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [11]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
checkpoint = keras.callbacks.ModelCheckpoint('model{epoch:08d}.h5', period=250) #save model after every 250 epochs
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=500,
          validation_split=0.2,callbacks=[checkpoint])
model.save('s2s.h5')
#model trained for 500 epochs

W0918 10:38:06.085654  2328 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0918 10:38:06.119651  2328 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0918 10:38:06.432111  2328 deprecation.py:323] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0918 10:38:09.183084  2328 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:986: 

Train on 8000 samples, validate on 2000 samples
Epoch 1/500
8000/8000 [==============================] - 86s 11ms/step - loss: 6.0603 - acc: 0.0643 - val_loss: 4.3884 - val_acc: 0.0800
Epoch 2/500
8000/8000 [==============================] - 91s 11ms/step - loss: 4.1974 - acc: 0.1184 - val_loss: 4.1638 - val_acc: 0.1200
Epoch 3/500
8000/8000 [==============================] - 90s 11ms/step - loss: 3.9813 - acc: 0.1199 - val_loss: 3.8941 - val_acc: 0.1200
Epoch 4/500
8000/8000 [==============================] - 99s 12ms/step - loss: 3.6040 - acc: 0.1959 - val_loss: 3.4926 - val_acc: 0.2708
Epoch 5/500
8000/8000 [==============================] - 101s 13ms/step - loss: 3.2253 - acc: 0.3882 - val_loss: 3.1716 - val_acc: 0.5120
Epoch 6/500
8000/8000 [==============================] - 104s 13ms/step - loss: 2.8892 - acc: 0.5118 - val_loss: 2.8521 - val_acc: 0.5128
Epoch 7/500
8000/8000 [==============================] - 68s 8ms/step - loss: 2.5694 - acc: 0.5547 - val_loss: 2.5873 - val_acc:

8000/8000 [==============================] - 109s 14ms/step - loss: 1.1642 - acc: 0.7648 - val_loss: 1.6437 - val_acc: 0.7378
Epoch 61/500
8000/8000 [==============================] - 85s 11ms/step - loss: 1.1595 - acc: 0.7648 - val_loss: 1.6477 - val_acc: 0.7380
Epoch 62/500
8000/8000 [==============================] - 95s 12ms/step - loss: 1.1545 - acc: 0.7651 - val_loss: 1.6655 - val_acc: 0.7381
Epoch 63/500
8000/8000 [==============================] - 117s 15ms/step - loss: 1.1499 - acc: 0.7649 - val_loss: 1.6614 - val_acc: 0.7383
Epoch 64/500
8000/8000 [==============================] - 103s 13ms/step - loss: 1.1448 - acc: 0.7651 - val_loss: 1.6764 - val_acc: 0.7385
Epoch 65/500
8000/8000 [==============================] - 85s 11ms/step - loss: 1.1396 - acc: 0.7652 - val_loss: 1.6747 - val_acc: 0.7381
Epoch 66/500
8000/8000 [==============================] - 75s 9ms/step - loss: 1.1346 - acc: 0.7651 - val_loss: 1.6864 - val_acc: 0.7404
Epoch 67/500
8000/8000 [=====================

C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.133965). Check your callbacks.
  % delta_t_median)


 256/8000 [..............................] - ETA: 4:02 - loss: 1.1173 - acc: 0.7702

...

8000/8000 [==============================] - 207s 26ms/step - loss: 0.1555 - acc: 0.9168 - val_loss: 2.4128 - val_acc: 0.7412
Epoch 440/500
8000/8000 [==============================] - 174s 22ms/step - loss: 0.1539 - acc: 0.9172 - val_loss: 2.4101 - val_acc: 0.7412
Epoch 441/500
8000/8000 [==============================] - 148s 18ms/step - loss: 0.1527 - acc: 0.9173 - val_loss: 2.4077 - val_acc: 0.7417
Epoch 442/500
8000/8000 [==============================] - 93s 12ms/step - loss: 0.1519 - acc: 0.9175 - val_loss: 2.4126 - val_acc: 0.7414
Epoch 443/500
8000/8000 [==============================] - 87s 11ms/step - loss: 0.1515 - acc: 0.9173 - val_loss: 2.4166 - val_acc: 0.7409
Epoch 444/500
8000/8000 [==============================] - 236s 29ms/step - loss: 0.1498 - acc: 0.9175 - val_loss: 2.4159 - val_acc: 0.7408
Epoch 445/500
8000/8000 [==============================] - 84s 11ms/step - loss: 0.1493 - acc: 0.9176 - val_loss: 2.4146 - val_acc: 0.7419
Epoch 446/500
8000/8000 [============

C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.784320). Check your callbacks.
  % delta_t_median)


 256/8000 [..............................] - ETA: 1:40 - loss: 0.1212 - acc: 0.9189

C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.392659). Check your callbacks.
  % delta_t_median)


8000/8000 [==============================] - 84s 10ms/step - loss: 0.1229 - acc: 0.9197 - val_loss: 2.4425 - val_acc: 0.7408
Epoch 476/500
8000/8000 [==============================] - 134s 17ms/step - loss: 0.1226 - acc: 0.9196 - val_loss: 2.4412 - val_acc: 0.7409
Epoch 477/500
8000/8000 [==============================] - 118s 15ms/step - loss: 0.1215 - acc: 0.9198 - val_loss: 2.4424 - val_acc: 0.7410
Epoch 478/500
8000/8000 [==============================] - 122s 15ms/step - loss: 0.1209 - acc: 0.9198 - val_loss: 2.4369 - val_acc: 0.7414
Epoch 479/500
8000/8000 [==============================] - 236s 30ms/step - loss: 0.1199 - acc: 0.9199 - val_loss: 2.4409 - val_acc: 0.7409
Epoch 480/500
8000/8000 [==============================] - 132s 16ms/step - loss: 0.1191 - acc: 0.9200 - val_loss: 2.4397 - val_acc: 0.7410
Epoch 481/500
8000/8000 [==============================] - 118s 15ms/step - loss: 0.1189 - acc: 0.9198 - val_loss: 2.4434 - val_acc: 0.7407
Epoch 482/500
8000/8000 [==========

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14265411717301435939
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3153238425
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15712725967867260402
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [12]:
import tensorflow as tf
tf.test.is_gpu_available()

True